[nn.Sequential]  
각 레이어를 데이터가 순차적으로 지나갈 때 사용  
여러 nn.Module을 한 컨테이너에 집어넣고 한 번에 돌리는 방법  

In [1]:
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, class_num, filter_num, num):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=5, kernel_size=5,
                      stride=1, padding = 2),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5,
                     stride=1, padding=2),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(out_channels*num*num, class_num)
        self.dropout = nn.Dropout()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.fc(x)
        return x

make_sequential() 이용한 방법

In [2]:
def make_sequential(in_channels, out_channels, *args, **kwargs):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(*args, **kwargs))

In [3]:
class CNNModel(nn.Module):
    def __init__(self, class_num, filter_num, num):
        super().__init__()
        self.layer1 = make_sequential(1, 32, kernel_size=5, stride=1, padding=2)
        self.layer2 = make_sequential(32, 64, kernel_size=5, stride=1, padding=2)
        self.fc = nn.Linear(out_channels*num*num, class_num)
        self.dropout = nn.Dropout()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.fc(x)
        return x

array 크기에 따라 자동으로 지정 가능

In [5]:
class CNNModel(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2, 
                 class_num, filter_num, num):
        super().__init__()
        self.layer_size = [in_channels, out_channels1, out_channels2]
        layers = [
            make_sequential(
                in_channels, out_channels, kernel_size=5, stride=5, padding=2)\
            for in_channels, out_channels in zip(
                self.layer_size, self.layer_size[1:])
        ]
        self.encoder = nn.Sequential(*layers)
        self.fc = nn.Linear(out_channels*num*num, class_num)
        self.dropout = nn.Dropout()
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc(x)
        return x

[nn.ModuleList]  
nn.Module을 리스트로 정리하는 방법  
각 레이어를 리스트에 전달하고 레이어의 iterator를 만듦

In [7]:
# 1.
class CNNModel(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2, 
                 class_num, filter_num, num):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc = nn.Linear(out_channels*num*num, class_num)
        self.dropout = nn.Dropout()
        
        layers = [self.conv1, self.batchnorm1, self.relu1, self.maxpool1,
                 self.conv2, self.batchnorm2, self.relu2, self.maxpool2,
                 self.fc, self.dropout]
        
        self.module_list = nn.ModuleList(layers)
        
    def forward(self, x):
        for layer in self.module_list:
            x = f(x)
        return x

In [8]:
# 2. text classification cnn pytorch. 
# 커널의 region size를 받아 nn.Conv2d에 넣는 형식

class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, num_class):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.kernel_sizes = kernel_sizes
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.ModuleList([
            nn.Conv2d(in_channels, out_channels, (kernel, embedding_dim))\
            for kernel in kernel_sizes
        ])
        self.dropout = nn.Dropout()
        self.fc = nn.Linear(len(kernel_sizes)*out_channels, num_class)